In [6]:
from maxatac.utilities.genome_tools import RandomRegionsGenerator
from maxatac.utilities.genome_tools import build_chrom_sizes_dict
import types


## Initialize the RandomRegionsGenerator

The generator takes as input:

<pre>
chrom_sizes_dict,
chrom_pool_size,
region_length,
sequence,
average,
meta_table,
input_channels,
bp_resolution=BP_RESOLUTION,
method="length"
</pre>

In [7]:
args = types.SimpleNamespace()

args.chrom_pool_size = 1000
args.region_length = 1024
args.sequence = "/Users/caz3so/scratch/maxATAC/data/genome_inf/hg38/hg38.2bit"
args.average = "/Users/caz3so/scratch/maxATAC/data/genome_inf/hg38/hg38_zeroes_average_file.bw"
args.meta_table = "/Users/caz3so/scratch/maxATAC/analysis/20201228_maxATAC_interactive/20201230_maxATAC_CTCF_TEST_META.tsv"
args.input_channels = 6
args.bp_resolution = 1
args.method = "length"
args.chrom_sizes = "/Users/caz3so/scratch/maxATAC/data/genome_inf/hg38/hg38.chrom.sizes"

In [9]:
RR_gen = RandomRegionsGenerator(build_chrom_sizes_dict(["chr1"], args.chrom_sizes),
                                chrom_pool_size=args.chrom_pool_size,
                                region_length=args.region_length,
                                sequence=args.sequence,
                                average=args.average,
                                meta_table=args.meta_table,
                                input_channels=args.input_channels,
                                bp_resolution=args.bp_resolution,
                                method=args.method)

In [13]:
RR_gen.chrom_pool[0:10]

[('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422),
 ('chr1', 248956422)]

In [14]:
RR_gen.MetaDF[0:10]

,Cell_Line,ATAC_Signal_File,TF,Binding_File,ATAC_Peaks,CHIP_Peaks
0,A549,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
1,GM12878,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
2,GM23338,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
3,HEK293,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
4,HepG2,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
5,IMR-90,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
6,K562,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...
7,MCF-7,/Users/caz3so/scratch/20201127_maxATAC_refined...,CTCF,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...,/Users/caz3so/scratch/maxATAC/analysis/2020122...


In [15]:
RR_gen.get_region()

('chr1', 42389921, 42390945)

In [17]:
next(RR_gen.BatchGenerator(n_rand=1))

(array([[[1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         ...,
         [0., 0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.]]]),
 array([[0., 0., 0., ..., 0., 0., 0.]]))

In [ ]:
class RandomRegionsGenerator(object):
    """
    This class will generate a pool of random regions

    The RandomRegionsGenerator will generate a random region of interest based
    based on the input reference genome chrom sizes, the desired size of the
    chromosome pool, the input length, and the method used to calculate the
    frequency of chromosome examples.

    Args
    ----
    chrom_sizes_dict (dict):
        A dictionary of chromosome sizes.
    chrom_pool_size (int):
        The size of the chromosome pool to output.
    region_length (int):
        The size of the regions to generate.
    sequence (str):
        The 2bit DNA sequence path.
    average (str):
        The average ATAC-seq sequence path.
    meta_table (str):
        Path to the meta run file.
    input_channels (int):
        The number of input channels.
    bp_resolution (int):
        The resolution of the data to use.
    method (str): optional
        Method for calculating chromosome frequencies.

    Attributes
    ----------
    chrom_sizes_dict (dict):
        A dictionary of chromosome sizes filtered for chromosomes of interest.
    chrom_pool_size (int):
        The size of the chromosome pool to output.
    region_length (int):
        The size of the regions to generate.
    method (str):
        Method for calculating chromosome frequencies.
    chrom_pool (array):
        Chromosome pool that is adjusted by frequency.

    Methods
    -------
    get_region()
        Gets a random region from the available pool

    """

    def __init__(
            self,
            chrom_sizes_dict,
            chrom_pool_size,
            region_length,
            sequence,
            average,
            meta_table,
            input_channels,
            bp_resolution=BP_RESOLUTION,
            method="length"
    ):
        self.chrom_sizes_dict = chrom_sizes_dict
        self.chrom_pool_size = chrom_pool_size
        self.region_length = region_length
        self.__idx = 0
        self.method = method
        self.sequence = sequence
        self.average = average
        self.MetaPath = meta_table
        self.input_channels = input_channels
        self.bp_resolution = bp_resolution

        self.chrom_pool = self.__get_chrom_frequencies()
        self.MetaDF = self.ImportMeta()
        self.CellTypes = self.get_CellTypes()
        self.TranscriptionFactor = self.get_TranscriptionFactor()

    def ImportMeta(self):
        return pd.read_csv(self.MetaPath, sep='\t', header=0, index_col=None)

    def get_CellTypes(self):
        return self.MetaDF["Cell_Line"].unique().tolist()

    def get_TranscriptionFactor(self):
        return self.MetaDF["TF"].unique()[0]

    def get_input_matrix(self,
                         rows,
                         cols,
                         batch_size,  # make sure that cols % batch_size == 0
                         signal_stream,
                         average_stream,
                         sequence_stream,
                         bp_order,
                         chrom,
                         start,  # end - start = cols
                         end,
                         reshape=True,

                         ):

        input_matrix = np.zeros((rows, cols))

        for n, bp in enumerate(bp_order):
            input_matrix[n, :] = get_one_hot_encoded(
                sequence_stream.sequence(chrom, start, end),
                bp
            )

        signal_array = np.array(signal_stream.values(chrom, start, end))
        avg_array = np.array(average_stream.values(chrom, start, end))
        input_matrix[4, :] = signal_array
        input_matrix[5, :] = input_matrix[4, :] - avg_array
        input_matrix = input_matrix.T

        if reshape:
            input_matrix = np.reshape(
                input_matrix,
                (batch_size, round(cols / batch_size), rows)
            )

        return input_matrix

    def __get_chrom_frequencies(self):
        """
        Generate an array of chromosome frequencies

        The frequencies will be used to build the pool of regions that
        we pull random regions from. The frequencies are determined by
        the method attribute. The methods are length or proportion.

        The length method will generate the frequencies of examples in
        the pool based on the length of the chromosomes in total.

        The proportionmethod will generate a pool that has chromosome
        frequencies equel to the chromosome pools size divided by the
        number of chromosomes.

        Returns
        -------
        labels (list):
            A list of chromosome names the size of the desired pool.

        """

        if self.method == "length":
            sum_lengths = sum(self.chrom_sizes_dict.values())

            frequencies = {
                chrom_name: round(chrom_len / sum_lengths * self.chrom_pool_size)
                for chrom_name, chrom_len in self.chrom_sizes_dict.items()
            }

        else:
            chrom_number = len(self.chrom_sizes_dict.values())

            frequencies = {
                chrom_name: round(self.chrom_pool_size / chrom_number)
                for chrom_name, chrom_len in self.chrom_sizes_dict.items()
            }

        labels = []

        for k, v in frequencies.items():
            labels += [(k, self.chrom_sizes_dict[k])] * v
        random.shuffle(labels)

        return labels

    def get_region(self):
        """
        Get a random region from the chromosome pool.

        Returns
        -------
        chrom_name (str):
            The chromosome string
        start (int):
            The region start
        end (int):
            The region end

        """

        # If the __idx reaches the pool size before enough samples are selected
        # shuffle and set to 0
        if self.__idx == self.chrom_pool_size:
            random.shuffle(self.chrom_pool)
            self.__idx = 0

        chrom_name, chrom_length = self.chrom_pool[self.__idx]
        self.__idx += 1

        start = round(
            random.randint(
                0,
                chrom_length - self.region_length
            )
        )

        end = start + self.region_length

        return chrom_name, start, end

    def BatchGenerator(self, n_rand):
        while True:
            inputs_batch, targets_batch = [], []
            for idx in range(n_rand):
                cell_line = random.choice(self.CellTypes)  # Randomly select a cell line

                chrom_name, start, end = self.get_region()  # returns random region (chrom_name, start, end)

                meta_row = self.MetaDF[self.MetaDF['Cell_Line'] == cell_line]

                meta_row = meta_row.reset_index(drop=True)

                signal = meta_row.loc[0, 'ATAC_Signal_File']

                binding = meta_row.loc[0, 'Binding_File']

                with \
                        load_bigwig(self.average) as average_stream, \
                        load_2bit(self.sequence) as sequence_stream, \
                        load_bigwig(signal) as signal_stream, \
                        load_bigwig(binding) as binding_stream:

                    input_matrix = self.get_input_matrix(
                        rows=self.input_channels,
                        cols=self.region_length,
                        batch_size=1,  # we will combine into batch later
                        reshape=False,
                        bp_order=["A", "C", "G", "T"],
                        signal_stream=signal_stream,
                        average_stream=average_stream,
                        sequence_stream=sequence_stream,
                        chrom=chrom_name,
                        start=start,
                        end=end
                    )
                    inputs_batch.append(input_matrix)
                    target_vector = np.array(binding_stream.values(chrom_name, start, end)).T
                    target_vector = np.nan_to_num(target_vector, 0.0)
                    n_bins = int(target_vector.shape[0] / self.bp_resolution)
                    split_targets = np.array(np.split(target_vector, n_bins, axis=0))
                    bin_sums = np.sum(split_targets, axis=1)
                    bin_vector = np.where(bin_sums > 0.5 * self.bp_resolution, 1.0, 0.0)
                    targets_batch.append(bin_vector)

            yield (np.array(inputs_batch), np.array(targets_batch))
